In [15]:
import numpy as np
import pandas as pd
import urllib.request as ureq
import json

evic_table = pd.read_csv("data/Eviction_Notices.csv", low_memory=False)
url = "https://geo.fcc.gov/api/census/area?"
fips_arr = []

def parseLocation(ln):
    return ln[7:-1].split(" ")

In [ ]:
for i, row in evic_table.iterrows():
    loc = parseLocation(str(row['Location']))
    if (len(loc) > 1):
        try:
            res = json.loads(ureq.urlopen(url + "lon=" + loc[0] + "&lat=" + loc[1]).read())
        except HTTPError as e:
            print 'Failure: ' + str(e.reason)
            fips_arr.append('')
        else:
            fips = res['results'][0]['block_fips'][1:11]
            fips_arr.append(fips)
    else:
        fips_arr.append('')

In [46]:
import datetime as dt
evic_table['fips code'] = pd.Series(fips_arr)
for i, row in evic_table.iterrows():
    evic_table.at[i, "File Date"] = dt.datetime.strptime(row['File Date'],"%m/%d/%y").strftime("%m-%d-%Y")
evic_table.to_csv("data/Evictions_fips.csv")